# Generate data used to test integration of EddyMotionEstimator

This notebook provides all code for the generation of files used in `test_integration.py` to test the `EddyMotionEstimator` using the trivial b0 model. In brief, it consists of:

1. Estimation of motion parameters from a fMRI recording obtained on OpenNeuro

2. Extraction of a diffusion-free b0 volume from a diffusion MRI scan (`b0.nii.gz`) and creation of the gradient table in RAS+B format (`gradients.tsv`)

3. Creation of (1) a 4D Nifti image (`b0.moving.nii.gz`) that contains a serie of 20 diffusion-free b0 volumes transformed using the motion parameters estimated on the last 20 frames of the fMRI recording, and (2) a new gradient table `gradients.moving.tsv`. Each of the 20 transforms are saved in ITK `.tfm` format.


## Notes
* The fMRI recording (`sub-01_task-mixedgamblestask_run-01_bold.nii.gz`) is not provided directy in `eddymotion/tests/data` but can be easily get from https://openneuro.org/datasets/ds000005/versions/00001. Please download it before going further in this notebook. but should be s is not included
* The notebook also assumes that a (`sub-01_dwi.nii.gz`, `sub-01_dwi.bval`, `sub-01_dwi.bvec`) file set is included inside `eddymotion/tests/data`.

In [1]:
import os
from glob import glob
from pathlib import Path
import subprocess
import nibabel
import numpy as np
from nibabel.eulerangles import euler2mat
from nibabel.affines import from_matvec
import nitransforms.linear as nitl

In [2]:
notebookdir = Path(os.path.dirname(os.path.realpath("__file__")))

## Estimate motion parameters from a BOLD fMRI using AFNI `3dvolreg`

In [3]:
datadir = Path((notebookdir.parents[1] / 'eddymotion' / 'tests' / 'data'))
bold_file = datadir / 'sub-01_task-mixedgamblestask_run-01_bold.nii.gz'
print(bold_file)

/Users/sebastientourbier/Softwares/forks/EddyMotionCorrection/eddymotion/tests/data/sub-01_task-mixedgamblestask_run-01_bold.nii.gz


In [4]:
def getNumberOfTimeFrames(bold_file):
    """Get the number of time frames in the fMRI recording."""
    _img = nibabel.load(str(bold_file))
    return _img.get_fdata().shape[3]

In [5]:
def afniEstimateMotionParameters(bold_file, out_file, n):
    """Call AFNI 3dvolreg to register each 3D sub-brick from the input fMRI to the n-th base brick."""
    
    # Get output filename without .nii.gz extension and make path for 
    # output motion parameters file.
    # (Path .stem was only able to remove the first extension (.gz))
    out_filename = os.path.basename(str(out_file))
    index_of_dot = out_filename.index('.')
    out_params_filename = f'{out_filename[:index_of_dot]}.motionparams.1d'
    out_params_file = os.path.join(out_file.parent, out_params_filename)
    
    cmd = ['3dvolreg']
    cmd.append('-prefix')
    cmd.append(f'{str(out_file)}')
    cmd.append('-base')
    cmd.append(str(n))
    cmd.append('-twopass')  # Make the alignment more precise
    cmd.append('-dfile')
    cmd.append(f'{out_params_file}')
    cmd.append(str(bold_file))

    print(f'Command to register each 3D sub-brick from the input fMRI to the n-th base brick.:\n {" ".join(cmd)}')
    try:
        p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        output, err = p.communicate()
        rc = p.returncode
    except Exception as e:
        print(e)
        return 1
    return rc, output, err, out_params_file

In [6]:
number_timepoints = getNumberOfTimeFrames(bold_file)
n = int(0.5 * number_timepoints) # Take the middle frame as reference
out_prefix = datadir / 'sub-01_task-mixedgamblestask_run-01_desc-motioncorr_bold.nii.gz'
rc, output, err, afni_params_file = afniEstimateMotionParameters(bold_file, out_prefix, n)

Command to register each 3D sub-brick from the input fMRI to the n-th base brick.:
 3dvolreg -prefix /Users/sebastientourbier/Softwares/forks/EddyMotionCorrection/eddymotion/tests/data/sub-01_task-mixedgamblestask_run-01_desc-motioncorr_bold.nii.gz -base 120 -twopass -dfile /Users/sebastientourbier/Softwares/forks/EddyMotionCorrection/eddymotion/tests/data/sub-01_task-mixedgamblestask_run-01_desc-motioncorr_bold.motionparams.1d /Users/sebastientourbier/Softwares/forks/EddyMotionCorrection/eddymotion/tests/data/sub-01_task-mixedgamblestask_run-01_bold.nii.gz


In [7]:
print(f'Output:\n{err.decode("utf-8")}')

Output:
++ 3dvolreg: AFNI version=AFNI_21.3.05 (Nov  7 2021) [64-bit]
++ Authored by: RW Cox
++ Coarse del was 10, replaced with 3
++ Max displacement in automask = 0.63 (mm) at sub-brick 0
++ Max delta displ  in automask = 0.15 (mm) at sub-brick 215



## Extract B0 from diffusion and create RAS+B gradient table

In [8]:
dwi_file = datadir / 'sub-01_dwi.nii.gz'
fbval = datadir / 'sub-01_dwi.bval'
fbvec = datadir / 'sub-01_dwi.bvec'

out_rasbn_file = datadir / 'gradients.tsv'
out_b0_file = datadir / 'b0.nii.gz'

In [9]:
def extract_diffusion_b0(dwi_file, out_b0_file):
    _img = nibabel.load(str(dwi_file))
    b0_data = _img.get_fdata()[..., 0]
    _b0_img = nibabel.Nifti1Image(b0_data, affine=_img.affine)
    print(_b0_img)
    nibabel.save(_b0_img, out_b0_file)

In [10]:
extract_diffusion_b0(dwi_file, out_b0_file)

<class 'nibabel.nifti1.Nifti1Image'>
data shape (128, 128, 66)
affine: 
[[ -2.           0.          -0.         126.84400177]
 [ -0.           2.          -0.         -94.82400513]
 [  0.           0.           2.         -26.26519966]
 [  0.           0.           0.           1.        ]]
metadata:
<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b''
dim_info        : 0
dim             : [  3 128 128  66   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float64
bitpix          : 64
slice_start     : 0
pixdim          : [-1.  2.  2.  2.  1.  1.  1.  1.]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 0
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax       

In [11]:
def fslgrad2rasb(dwi_file, fbval, fbvec, out_rasbn_file):
    """Save gradient table in RAS+B format taking as input the DWI with FSL `.bval` and `.bvec`."""
    import numpy as np
    from nibabel import load
    from dipy.io import read_bvals_bvecs
    
    # Read / Load
    img = load(str(dwi_file))
    bvals, bvecs = read_bvals_bvecs(str(fbval), str(fbvec))
    
    # Apply the affine transform to bvecs
    bvecs_tr = np.matmul(img.affine[:3,:3], bvecs.T).T
    
    # Normalize the bvecs
    norm = np.sum(bvecs_tr**2, axis=1)
    bvecs_tr_norm = np.zeros_like(bvecs_tr)
    for i in range(bvecs_tr.shape[0]):
        bvecs_tr_norm[i, :] = bvecs_tr[i, :] / norm[i] 
    # Handles bzeros
    bvecs_tr_norm = np.nan_to_num(bvecs_tr_norm)
    
    rasbn = np.c_[bvecs_tr_norm, bvals]
    
    # Save Nx4 numpy matrix in TSV text file
    np.savetxt(fname=str(out_rasbn_file),
               delimiter="\t",
               X=rasbn)
    
    return rasbn
    

In [12]:
rasbn = fslgrad2rasb(dwi_file, fbval, fbvec, out_rasbn_file)

<ipython-input-11-cb78b93f219c>:18: RuntimeWarning: invalid value encountered in true_divide
  bvecs_tr_norm[i, :] = bvecs_tr[i, :] / norm[i]


In [13]:
def create_moving_b0s_rasbn_gradient(rasbn_file, start_index, nb_of_gradient):
    rasbn = np.genfromtxt(fname=out_rasbn_file, delimiter="\t", skip_header=0)
    moving_b0s_rasbn = np.zeros((nb_of_gradient,4))
    moving_b0s_rasbn[0:nb_of_gradient, ...] = rasbn[start_index:start_index+nb_of_gradient, ...]
    # Save Nx4 numpy matrix in TSV text file
    np.savetxt(fname=str(rasbn_file.parent / 'gradients.moving.tsv'),
               delimiter="\t",
               X=moving_b0s_rasbn)
    return moving_b0s_rasbn

In [14]:
moving_b0s_rasbn = create_moving_b0s_rasbn_gradient(out_rasbn_file, 6, 20)
print(moving_b0s_rasbn)
print(moving_b0s_rasbn.shape)

[[ 1.80353913e-01  4.66122441e-01  0.00000000e+00  1.00000000e+03]
 [-1.27439466e-01  2.82365876e-01  3.92313651e-01  1.00000000e+03]
 [ 4.30191553e-01 -2.31833775e-01  1.04924769e-01  1.00000000e+03]
 [-1.53638121e-01 -3.83344627e-01  2.82253746e-01  1.00000000e+03]
 [-3.67928622e-01  6.49873924e-03  3.38434344e-01  1.00000000e+03]
 [ 2.66066783e-01  1.71543057e-01  3.87097161e-01  1.00000000e+03]
 [ 8.84487882e-02  4.82220794e-01  9.74435802e-02  1.00000000e+03]
 [ 3.85795133e-01  8.15623952e-02  3.07735418e-01  1.00000000e+03]
 [ 3.95176644e-02 -4.98222706e-01 -1.80080496e-02  1.00000000e+03]
 [ 5.45186999e-02 -4.60157834e-01 -1.88064506e-01  1.00000000e+03]
 [ 1.51083700e-01 -3.89715903e-01 -2.74652157e-01  1.00000000e+03]
 [-2.32012761e-01 -2.30012651e-01  3.78520819e-01  1.00000000e+03]
 [-2.32029468e-01  4.19553283e-01  1.42018036e-01  1.00000000e+03]
 [-2.64278535e-01  6.99413891e-03 -4.24144567e-01  1.00000000e+03]
 [-4.12452980e-01 -2.69969224e-01 -8.34904821e-02  1.00000000e

## Create moving b0 volumes using the motion parameters estimated from fMRI

In [15]:
def create_moving_b0s_and_transforms(b0_file, params_file, indices):
    """Move the b0 volume according to motion parameters estimated from a list of time frame indices."""
    
    # Get b0 filename without .nii.gz extension and make path from it
    b0_filename = os.path.basename(str(b0_file))
    index_of_dot = b0_filename.index('.')
    
    # Get b0 affine transform
    b0_affine = nibabel.load(str(b0_file)).affine
    
    # Initialize the matrix containing all b0 volumes
    nb_of_moving_b0s = len(indices) + 1
    x_shape, y_shape, z_shape = nibabel.load(str(b0_file)).get_fdata().shape
    moving_b0s_data = np.zeros((x_shape, y_shape, z_shape, nb_of_moving_b0s))
    
    # Add first b0
    moving_b0s_data[..., 0] = nibabel.load(str(b0_file)).get_fdata()
    
    # Read the motion parameter file generated by 3dvolreg
    with open(params_file) as file:
        lines = file.readlines()
        xfm_files = []
        xforms = []
        cnt=1
        for i in indices:
            # Extract the parameters for time frame i
            formatted_line = lines[i].split()
            transform_params = np.array(formatted_line).astype(float)
            print(transform_params)
            
            # Create a nitransforms object
            T = from_matvec(
                euler2mat(
                    x=transform_params[1],
                    y=transform_params[2],
                    z=transform_params[3],
                ),
                [
                    transform_params[4],
                    transform_params[5],
                    transform_params[6]
                ]
            )
            xfm = nitl.Affine(T, reference=b0_file)
            
            # Apply the transform and save into a nifti file
            moving_b0_filename = f'{b0_filename[:index_of_dot]}.motion-{i}.nii.gz'
            moving_b0_file = b0_file.parent / moving_b0_filename
            (~xfm).apply(b0_file, reference=b0_file).to_filename(moving_b0_file)
            
            # Save the transform
            xfm_filename = f'{b0_filename[:index_of_dot]}.motion-{i}.tfm'
            xfm_file = b0_file.parent / xfm_filename
            xfm.to_filename(xfm_file, fmt="itk")
            
            moving_b0s_data[..., cnt] = nibabel.load(str(moving_b0_file)).get_fdata().copy()
            os.remove(moving_b0_file)

            xfm_files.append(xfm_file)
            xforms.append(xfm)
            cnt += 1
            
        moving_b0s_img = nibabel.Nifti1Image(moving_b0s_data, affine=b0_affine)
        moving_b0s_filename = f'{b0_filename[:index_of_dot]}.moving.nii.gz'
        moving_b0s_file = b0_file.parent / moving_b0s_filename
        moving_b0s_img.to_filename(str(moving_b0s_file))

        return xforms, xfm_files, moving_b0s_file

In [16]:
indices=np.arange(220,240)
print(indices)
xforms, xfm_files, moving_b0s_file = create_moving_b0s_and_transforms(
    out_b0_file, afni_params_file, indices
)

[220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237
 238 239]
[ 2.200e+02 -4.410e-02  1.570e-02  4.780e-02  2.840e-02 -6.820e-02
  1.140e-02  8.900e+00  8.682e+00]
[ 2.210e+02 -3.690e-02  7.600e-03  4.750e-02  2.610e-02 -5.710e-02
 -2.720e-02  8.782e+00  8.595e+00]
[ 2.220e+02 -4.050e-02  1.660e-02  5.230e-02  1.960e-02 -6.230e-02
  1.140e-02  8.784e+00  8.599e+00]
[ 2.230e+02 -4.140e-02  4.300e-03  3.870e-02  2.070e-02 -5.670e-02
 -3.700e-02  8.663e+00  8.481e+00]
[ 2.240e+02 -3.870e-02  2.570e-02  4.570e-02  2.050e-02 -5.860e-02
  2.450e-02  8.830e+00  8.636e+00]
[ 2.250e+02 -4.170e-02  5.700e-03  3.330e-02  3.490e-02 -5.530e-02
 -4.410e-02  8.415e+00  8.215e+00]
[ 2.260e+02 -4.650e-02  3.980e-02  3.780e-02  1.450e-02 -5.600e-02
  2.370e-02  8.328e+00  8.131e+00]
[ 2.270e+02 -5.420e-02  5.400e-03  3.400e-02  3.390e-02 -6.010e-02
 -5.500e-02  7.585e+00  7.300e+00]
[ 2.280e+02 -5.270e-02  2.790e-02  3.140e-02  2.180e-02 -6.360e-02
 -9.700e-03  8.115e+00  7.929e+00]